In [1]:
!nvidia-smi

Tue Feb 25 08:25:37 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.216.01             Driver Version: 535.216.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A5000               Off | 00000000:3B:00.0 Off |                  Off |
| 30%   21C    P8              12W / 230W |    207MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import penman

In [3]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [4]:
# import text_to_amr

In [5]:
# from huggingface_hub import snapshot_download
# from text_to_amr import TextToAMR

In [7]:
# # Gold-v2
# snapshot_download(
#     repo_id="abdiharyadi/mbart-en-id-smaller-indo-amr-parsing-translated-nafkhan",
#     local_dir=f"models/mbart-en-id-smaller-indo-amr-parsing-translated-nafkhan",
#     ignore_patterns=[
#         "*log*",
#         "*checkpoint*",
#     ]
# )

# Silver
# snapshot_download(
#     repo_id="abdiharyadi/indoamrbart-ft",
#     local_dir=f"{PROJECT_ROOT}/AMRBART-id/models",
#     allow_patterns=["*mbart-en-id-smaller-fted*"],
#     ignore_patterns=[
#         "*log*",
#         "*iter*",
#     ]
# )

# # Gold-v1
# snapshot_download(
#     repo_id="abdiharyadi/indoamrbart-mbart-double-ft",
#     local_dir=f"{PROJECT_ROOT}/AMRBART-id/models",
#     allow_patterns=["*indoamrbart-mbart-fted*"],
#     ignore_patterns=[
#         "*checkpoint*",
#         "*log*",
#     ]
# )

In [8]:
sentences = [
    "Saya suka makan ayam.",
    "Dia pergi ke pasar.",
    "Dia membeli ayam.",
    "Dia membeli ayam itu.",
    "Dia membeli ayam itu di pasar.",
    "Dia membeli ayam itu di pasar yang besar.",
    "Dia membeli ayam itu di pasar yang besar di Jakarta.",
    "Dia membeli ayam itu di pasar yang besar di Jakarta yang ramai.",
    "Dia membeli ayam itu di pasar yang besar di Jakarta yang ramai pada hari minggu.",
    "Dia membeli ayam itu di pasar yang besar di Jakarta yang ramai pada hari minggu yang cerah.",
    "Dia membeli ayam itu di pasar yang besar di Jakarta yang ramai pada hari minggu yang cerah di pagi hari.",
    "Dia membeli ayam itu di pasar yang besar di Jakarta yang ramai pada hari minggu yang cerah di pagi hari yang cerah.",
    ]

In [9]:
from model_interface.tokenization_bart import AMRBartTokenizer

2025-02-25 08:26:16.612883: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-25 08:26:16.615763: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-25 08:26:16.668353: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-25 08:26:17.437131: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [10]:
from transformers import MBartForConditionalGeneration, AutoConfig
import torch
import penman

class TextToAMRSan:
    def __init__(self, model_path="./models/mbart-en-id-smaller-indo-amr-parsing-translated-nafkhan"):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        
        # Load config
        self.config = AutoConfig.from_pretrained(model_path)
        
        # Initialize tokenizer with correct parameters
        self.tokenizer = AMRBartTokenizer.from_pretrained(
            model_path,
            use_fast=False
        )
        
        # Initialize model with config
        self.model = MBartForConditionalGeneration.from_pretrained(
            model_path,
            config=self.config
        ).to(self.device)
        
        # Set important parameters
        self.max_src_length = 400
        self.max_gen_length = 1024
        self.num_beams = 5
        
    def __call__(self, sentence: str) -> penman.Graph:
        # Prepare input with AMR special tokens
        raw_txt_ids = self.tokenizer(
            sentence,
            max_length=self.max_src_length,
            padding=False,
            truncation=True
        )["input_ids"]
        
        # Add AMR special tokens
        txt_ids = [raw_txt_ids[:self.max_src_length-3] + [
            self.tokenizer.amr_bos_token_id,
            self.tokenizer.mask_token_id,
            self.tokenizer.amr_eos_token_id
        ]]
        
        # Pad and convert to tensor
        txt_ids = self.tokenizer.pad(
            {"input_ids": txt_ids},
            padding=True,
            return_tensors="pt"
        ).to(self.device)
        
        # Generate
        preds = self.model.generate(
            txt_ids["input_ids"],
            max_length=self.max_gen_length,
            num_beams=self.num_beams,
            use_cache=True,
            decoder_start_token_id=self.tokenizer.amr_bos_token_id,
            eos_token_id=self.tokenizer.amr_eos_token_id,
            pad_token_id=self.tokenizer.pad_token_id,
            no_repeat_ngram_size=0,
            min_length=0
        )
        
        # Decode AMR
        pred = preds[0]
        pred[0] = self.tokenizer.bos_token_id
        pred = [
            self.tokenizer.eos_token_id if itm == self.tokenizer.amr_eos_token_id else itm
            for itm in pred if itm != self.tokenizer.pad_token_id
        ]
        
        graph, status, (lin, backr) = self.tokenizer.decode_amr(
            pred,
            restore_name_ops=False
        )
        
        # Add metadata
        metadata = {
            "id": "0",
            "annotator": "TextToAMRSan",
            "snt": sentence
        }
        graph.metadata = metadata
        
        return graph

In [11]:
text_to_amr = TextToAMRSan()

# graph = text_to_amr(sentences[0])
# print("AMR Graph:")
# print(penman.encode(graph))

In [17]:
graphs = []

for i in range(len(sentences)):
    graph = text_to_amr(sentences[i])
    graphs.append(graph)
    print("AMR Graph:")
    print(penman.encode(graph))

AMR Graph:
# ::id 0
# ::annotator TextToAMRSan
# ::snt Saya suka makan ayam.
(z0 / suka-01
    :ARG0 (z1 / aku)
    :ARG1 (z2 / makan-01
              :ARG0 z1
              :ARG1 (z3 / ayam)))
AMR Graph:
# ::id 0
# ::annotator TextToAMRSan
# ::snt Dia pergi ke pasar.
(z0 / pergi-01
    :ARG0 (z1 / dia)
    :ARG4 (z2 / pasar))
AMR Graph:
# ::id 0
# ::annotator TextToAMRSan
# ::snt Dia membeli ayam.
(z0 / beli-01
    :ARG0 (z1 / dia)
    :ARG3 (z2 / ayam))
AMR Graph:
# ::id 0
# ::annotator TextToAMRSan
# ::snt Dia membeli ayam itu.
(z0 / beli-01
    :ARG0 (z1 / dia)
    :ARG1 (z2 / ayam
              :mod (z3 / itu)))
AMR Graph:
# ::id 0
# ::annotator TextToAMRSan
# ::snt Dia membeli ayam itu di pasar.
(z0 / beli-01
    :ARG0 (z1 / dia)
    :ARG1 (z2 / ayam
              :mod (z3 / itu))
    :ARG3 (z4 / pasar))
AMR Graph:
# ::id 0
# ::annotator TextToAMRSan
# ::snt Dia membeli ayam itu di pasar yang besar.
(z0 / beli-01
    :ARG0 (z1 / dia)
    :ARG1 (z2 / ayam)
    :ARG3 (z3 / pasar
  

In [46]:

class AMRAugmenter:
    def __init__(self, source='conceptnet', conceptnet_api="http://api.conceptnet.io"):
        self.source = source.lower()
        if self.source not in ['nltk', 'conceptnet']:
            raise ValueError("Source must be either 'nltk' or 'conceptnet'")
        self.conceptnet_api = conceptnet_api
        self.pred_error_prob = 0.3
        self.entity_error_prob = 0.3
        # Add new error probabilities
        self.circumstance_error_prob = 0.3
        self.discourse_error_prob = 0.3
        
        # Define circumstance roles that can be modified
        self.circumstance_roles = {
            ':time', ':location', ':manner', ':duration', 
            ':instrument', ':purpose', ':source', ':destination'
        }
        
        # Define discourse roles that can be modified
        self.discourse_roles = {
            ':cause', ':condition', ':concession', ':consequence',
            ':temporal-before', ':temporal-after', ':temporal-during'
        }
    
    def get_related_words(self, word):
        """Get related words based on selected source"""
        if self.source == 'nltk':
            return self._get_nltk_related_words(word)
        return self._get_conceptnet_related_words(word)
    
    def _get_nltk_related_words(self, word):
        """Get related words using NLTK WordNet"""
        related_words = []
        synsets = wordnet.synsets(word, lang='ind')
        
        for synset in synsets:
            # Add lemma names
            related_words.extend([lemma.name() for lemma in synset.lemmas(lang='ind')])
            # Add hypernyms
            for hypernym in synset.hypernyms():
                related_words.extend([lemma.name() for lemma in hypernym.lemmas(lang='ind')])
            # Add hyponyms
            for hyponym in synset.hyponyms():
                related_words.extend([lemma.name() for lemma in hyponym.lemmas(lang='ind')])
        
        return list(set(related_words))
    
    def _get_conceptnet_related_words(self, word):
        """Get related Indonesian words from ConceptNet"""
        url = f"{self.conceptnet_api}/c/id/{word}"
        response = requests.get(url).json()
        related_words = []
        
        for edge in response.get('edges', []):
            if edge['start']['language'] == 'id':
                related_words.append(edge['start']['label'])
            if edge['end']['language'] == 'id':
                related_words.append(edge['end']['label'])
                
        return list(set(related_words))

    def introduce_predicate_error(self, amr_graph):
        """Introduce errors in predicates"""
        modified_graph = amr_graph.copy()
        
        for node in modified_graph.nodes():
            if random.random() < self.pred_error_prob:
                if 'predicate' in modified_graph.nodes[node]:
                    current_pred = modified_graph.nodes[node]['predicate']
                    alternatives = self.get_related_words(current_pred)
                    if alternatives:
                        modified_graph.nodes[node]['predicate'] = random.choice(alternatives)
        
        return modified_graph

    def introduce_entity_error(self, amr_graph):
        """Introduce errors in entities"""
        modified_graph = amr_graph.copy()
        
        for node in modified_graph.nodes():
            if random.random() < self.entity_error_prob:
                if 'entity' in modified_graph.nodes[node]:
                    current_entity = modified_graph.nodes[node]['entity']
                    alternatives = self.get_related_words(current_entity)
                    if alternatives:
                        modified_graph.nodes[node]['entity'] = random.choice(alternatives)
        
        return modified_graph

    def introduce_circumstance_error(self, amr_graph):
        """Introduce errors in circumstantial information (time, location, etc.)"""
        modified_graph = amr_graph.copy()
        
        for source, target, data in modified_graph.edges(data=True):
            role = data.get('role', '')
            if role in self.circumstance_roles and random.random() < self.circumstance_error_prob:
                # Strategy 1: Swap with another circumstance of the same type
                edges_same_role = [(s, t, d) for (s, t, d) in modified_graph.edges(data=True) 
                                 if d.get('role') == role and (s, t) != (source, target)]
                if edges_same_role:
                    other_source, other_target, _ = random.choice(edges_same_role)
                    # Swap targets
                    modified_graph.remove_edge(source, target)
                    modified_graph.remove_edge(other_source, other_target)
                    modified_graph.add_edge(source, other_target, role=role)
                    modified_graph.add_edge(other_source, target, role=role)
                
                # Strategy 2: Change the circumstance type
                elif random.random() < 0.5:
                    new_role = random.choice(list(self.circumstance_roles - {role}))
                    modified_graph.remove_edge(source, target)
                    modified_graph.add_edge(source, target, role=new_role)

        return modified_graph

    def introduce_discourse_error(self, amr_graph):
        """Introduce errors in discourse links between statements"""
        modified_graph = amr_graph.copy()
        
        discourse_edges = [(s, t, d) for (s, t, d) in modified_graph.edges(data=True) 
                          if d.get('role') in self.discourse_roles]
        
        for source, target, data in discourse_edges:
            if random.random() < self.discourse_error_prob:
                current_role = data['role']
                
                # Strategy 1: Change discourse relation type
                if random.random() < 0.5:
                    new_role = random.choice(list(self.discourse_roles - {current_role}))
                    modified_graph.remove_edge(source, target)
                    modified_graph.add_edge(source, target, role=new_role)
                
                # Strategy 2: Reverse the direction of the relation
                else:
                    modified_graph.remove_edge(source, target)
                    modified_graph.add_edge(target, source, role=current_role)
        
        return modified_graph

    def nx_to_penman(self, nx_graph):
        """Convert NetworkX graph to Penman graph format"""
        triples = []
        instance_triples = []
        
        # First add instance triples
        for node in nx_graph.nodes():
            if 'predicate' in nx_graph.nodes[node]:
                instance_triples.append(Triple(source=str(node), 
                                            role=':instance',
                                            target=nx_graph.nodes[node]['predicate']))
            elif 'entity' in nx_graph.nodes[node]:
                instance_triples.append(Triple(source=str(node),
                                            role=':instance',
                                            target=nx_graph.nodes[node]['entity']))
        
        # Then add relation triples
        for source, target, data in nx_graph.edges(data=True):
            triples.append(Triple(source=str(source),
                                role=f":{data['role']}",
                                target=str(target)))
        
        # Combine instance and relation triples
        all_triples = instance_triples + triples
        
        # Create Penman graph
        return Graph(all_triples)

    def penman_to_nx(self, penman_graph):
        """Convert Penman graph to NetworkX format"""
        nx_graph = nx.DiGraph()
        
        # Process instance triples
        for triple in penman_graph.instances():
            node_id = triple.source
            if triple.target.startswith('pred_'):
                nx_graph.add_node(node_id, predicate=triple.target[5:])
            else:
                nx_graph.add_node(node_id, entity=triple.target)
        
        # Process relation triples
        for triple in penman_graph.edges():
            if not triple.role.startswith(':instance'):
                nx_graph.add_edge(triple.source, triple.target, 
                                role=triple.role[1:])  # Remove : prefix
        
        return nx_graph

    def augment_amr(self, amr_graph):
        """Main function to augment AMR graph with errors"""
        # Convert to NetworkX if input is Penman Graph
        if isinstance(amr_graph, Graph):
            nx_graph = self.penman_to_nx(amr_graph)
        else:
            nx_graph = amr_graph
            
        # Perform augmentation with all error types
        modified_graph = self.introduce_predicate_error(nx_graph)
        modified_graph = self.introduce_entity_error(modified_graph)
        modified_graph = self.introduce_circumstance_error(modified_graph)
        modified_graph = self.introduce_discourse_error(modified_graph)
        
        # Convert back to Penman if input was Penman
        if isinstance(amr_graph, Graph):
            return self.nx_to_penman(modified_graph)
        return modified_graph

    def visualize_graph(self, graph, title="AMR Graph"):
        """Visualize AMR graph with labels"""
        plt.figure(figsize=(12, 8))
        pos = nx.spring_layout(graph)
        
        # Draw nodes
        nx.draw_networkx_nodes(graph, pos, node_color='lightblue', 
                             node_size=2000, alpha=0.7)
        
        # Create node labels
        node_labels = {}
        for node in graph.nodes():
            label = []
            if 'predicate' in graph.nodes[node]:
                label.append(f"pred: {graph.nodes[node]['predicate']}")
            if 'entity' in graph.nodes[node]:
                label.append(f"ent: {graph.nodes[node]['entity']}")
            node_labels[node] = '\n'.join(label)
        
        # Draw edge labels
        edge_labels = nx.get_edge_attributes(graph, 'role')
        nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels)
        
        # Draw edges
        nx.draw_networkx_edges(graph, pos, edge_color='gray', arrows=True, 
                             arrowsize=20)
        
        # Draw labels
        nx.draw_networkx_labels(graph, pos, node_labels, font_size=10)
        
        plt.title(title)
        plt.axis('off')
        return plt

In [47]:
augmenter_nltk = AMRAugmenter(source='nltk')

In [48]:
from penman import Graph, Triple, encode, decode

In [49]:
import networkx as nx
import random
from nltk.corpus import wordnet
import matplotlib.pyplot as plt


In [50]:
graphs[6].triples

[('z0', ':instance', 'beli-01'),
 ('z0', ':ARG0', 'z1'),
 ('z1', ':instance', 'dia'),
 ('z0', ':ARG1', 'z2'),
 ('z2', ':instance', 'ayam'),
 ('z0', ':ARG3', 'z3'),
 ('z3', ':instance', 'pasar'),
 ('z4', ':ARG1', 'z3'),
 ('z4', ':instance', 'besar-01'),
 ('z3', ':location', 'z5'),
 ('z5', ':instance', 'kota'),
 ('z5', ':wiki', '"Jakarta"'),
 ('z5', ':name', 'z6'),
 ('z6', ':instance', 'nama'),
 ('z6', ':op1', '"Jakarta"')]

In [51]:
augmented_graph = augmenter_nltk.augment_amr(graphs[6])
augmented_graph

<Graph object (top=z0) at 134737383268576>

In [53]:
print(encode(graphs[6]))

# ::id 0
# ::annotator TextToAMRSan
# ::snt Dia membeli ayam itu di pasar yang besar di Jakarta.
(z0 / beli-01
    :ARG0 (z1 / dia)
    :ARG1 (z2 / ayam)
    :ARG3 (z3 / pasar
              :ARG1-of (z4 / besar-01)
              :location (z5 / kota
                            :wiki "Jakarta"
                            :name (z6 / nama
                                      :op1 "Jakarta"))))


In [52]:
print(encode(augmented_graph))

(z0 / beli-01
    :ARG0 (z1 / dia)
    :ARG1 (z2 / ayam_pedaging)
    :ARG3 (z3 / gedung
              :location (z5 / kota
                            :name (z6 / diagnosis))
              :ARG1-of (z4 / besar-01)))


In [54]:
from graphviz import Digraph

def visualize_amr_with_graphviz(triples):
    # Create a new directed graph
    dot = Digraph(comment='AMR Graph')
    dot.attr(rankdir='TB')  # Top to bottom layout
    
    # Add nodes and edges
    nodes = set()
    for source, relation, target in triples:
        # Add nodes if they don't exist
        if source not in nodes:
            dot.node(source, source)
            nodes.add(source)
        if target not in nodes:
            dot.node(target, target)
            nodes.add(target)
        
        # Add edge with relation label
        dot.edge(source, target, relation)
    
    # Display the graph
    return dot

In [55]:
graph_viz = visualize_amr_with_graphviz(augmented_graph.triples)
graph_viz1 = visualize_amr_with_graphviz(graphs[6].triples)

In [56]:
graph_viz.render('amr_graph', format='png', cleanup=True)

'amr_graph.png'

In [57]:
graph_viz1.render('amr_graph1', format='png', cleanup=True)

'amr_graph1.png'

In [45]:
for i in range(len(graphs)):
    augmented_graph = augmenter_nltk.augment_amr(graphs[i])
    graph_viz = visualize_amr_with_graphviz(augmented_graph.triples)
    graph_viz_original = visualize_amr_with_graphviz(graphs[i].triples)
    graph_viz_original.render(f'amr_graph_original_{i}', format='png', cleanup=True)
    graph_viz.render(f'amr_graph_{i}', format='png', cleanup=True)

In [58]:
from transformers import AutoModelForSeq2SeqLM, T5TokenizerFast
import torch
import os

class AMRToTextSan:
    def __init__(self, model_path):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        
        # Initialize tokenizer and model
        self.tokenizer = T5TokenizerFast.from_pretrained(
            os.path.join(model_path, 'tokenizer')
        )
        self.model = AutoModelForSeq2SeqLM.from_pretrained(
            os.path.join(model_path, 'model')
        ).to(self.device)
        
        # Set generation parameters
        self.max_seq_len_sent = 512
        self.num_beams = 5

    def __call__(self, amr_string: str) -> str:
        # Prepare input
        inputs = self.tokenizer(
            amr_string,
            max_length=self.max_seq_len_sent,
            padding=False,
            truncation=True,
            return_tensors="pt"
        ).to(self.device)
        
        # Generate
        outputs = self.model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            num_beams=self.num_beams,
            max_length=self.max_seq_len_sent,
            early_stopping=True,
            pad_token_id=self.tokenizer.pad_token_id,
            eos_token_id=self.tokenizer.eos_token_id
        )
        
        # Decode output
        generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return generated_text


In [59]:
amr_to_text = AMRToTextSan("best_model")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [60]:
generated_text = amr_to_text(encode(graphs[6]))
print("Generated text:")
print(generated_text)

generated_augmented_text = amr_to_text(encode(augmented_graph))
print("Generated augmented text:")
print(generated_augmented_text)

Generated text:
dia membeli ayam itu di pasar yang besar dia bernamaarg0-of z1-of z2-ayam di pasar besar-01-arg0-of z5-kota besar-01-wiki
Generated augmented text:
z4 besar-01:AR-0-of z1-dia :AR-0-of z1-dia :AR-0-of z2 ayam-pedaging :AR-0-of z4 besar-01:AR-0-of z4 besar-01:AR-0-of z4 besar-01:AR-0-of z4 besar-01


In [ ]:
for i in range(len(sentences)):
    graph = text_to_amr(sentences[i])
    print("AMR Graph:")
    a = penman.encode(graph)
    generated_text = amr_to_text(a)
    print("Generated:", generated_text)

# sent1 = sentences[0]
    
# # Text -> AMR -> Text
# amr_graph = text_to_amr(sent1)
# amr_string = penman.encode(amr_graph)
# generated_text = amr_to_text(amr_string)

# print("Original:", sent1)
# print("AMR:", amr_string)
# print("Generated:", generated_text)

AMR Graph:
# ::id 0
# ::annotator TextToAMRSan
# ::snt Balon itu ditiup Ilham
(z0 / gulir-01
    :ARG0 (z1 / orang
              :wiki -
              :name (z2 / nama
                        :op1 "Ilham"))
    :ARG1 (z3 / balon
              :mod (z4 / itu)))
Generated: balon itu ditiup oleh Ilham z0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0 orang di wiki----------------------------------------------------------------------------------------------------------------------------------------------------------------------
AMR Graph:
# ::id 0
# ::annotator TextToAMRSan
# ::snt Presiden Donald Trump menghabiskan bulan pertama masa jabatan keduanya dalam misi luar biasa 
(z0 / habis-01
    :ARG0 (z1 / orang
              :wiki "Donald_Trump"
              :name (z2 / nama
                        :op1 "Donald"
                        :op2 "Trump")
              :ARG0-of (z3 / punya-peran-org-91
                           :ARG2 (z4 / presiden)))
    :ARG1 (z5 / bulan
    

In [28]:
sent1 = sentences[1]
    
# Text -> AMR -> Text
amr_graph = text_to_amr(sent1)

amr_graph.metadata = {}

amr_string = penman.encode(amr_graph)


generated_text = amr_to_text(amr_string)

print("Original:", sent1)
print("AMR:", amr_string)
print("Generated:", generated_text)

Original: Sisa uang yang aku miliki saat ini hanya berjumlah sepuluh ribu rupiah
AMR: (z0 / sisa-01
    :ARG1 (z1 / uang
              :ARG1-of (z2 / punya-01
                           :ARG0 (z3 / aku)
                           :time (z4 / saat
                                     :mod (z5 / ini))))
    :ARG2 (z6 / kuantitas-moneter
              :quant 10
              :unit (z7 / dolar)
              :mod (z8 / hanya)))
Generated: z1, z2, z3, aku punya uang :arg1-of z2, z3, aku punya uang :arg1-of z6, kuantitas-moneter :quant 10-unit z7 dolar, z8 hanya saat ini.


In [24]:
sentences[1]

'Sisa uang yang aku miliki saat ini hanya berjumlah sepuluh ribu rupiah'

In [27]:
amr_graph.metadata = {}